In [3]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, vstack, hstack
from sklearn.preprocessing import MaxAbsScaler
from implicit.nearest_neighbours import TFIDFRecommender, ItemItemRecommender, CosineRecommender
from time import time
pd.set_option('display.max_columns', None)

from tqdm.notebook import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings('ignore')

In [4]:
# !pip install implicit

In [6]:
%%time
player_starts = pd.read_parquet('player_starts_train.parquet')
player_starts

CPU times: user 41.8 s, sys: 11.6 s, total: 53.4 s
Wall time: 44.7 s


,date,user_id,item_id,watch_time,is_autorized
0,2023-07-21 19:04:50+03:00,user_12964323,video_1042531,51,0
1,2023-07-21 02:02:41+03:00,user_16517,video_1707159,31,0
2,2023-07-21 22:00:47+03:00,user_15057892,video_1989987,9,0
3,2023-07-21 19:09:43+03:00,user_2846972,video_1356486,-1,0
4,2023-07-21 11:06:58+03:00,user_20517034,video_1380654,11,0
...,...,...,...,...,...
69954375,2023-08-21 02:51:53+03:00,user_15478739,video_1449287,291,1
69954376,2023-08-21 08:40:18+03:00,user_25783543,video_1423321,2,0
69954377,2023-08-21 05:19:55+03:00,user_3507470,video_464555,261,0
69954378,2023-08-21 12:32:32+03:00,user_13128840,video_420973,21,0


In [7]:
sample_submission = pd.read_csv('sample_submission.csv', sep=',', index_col=None)
sample_submission

,user_id,recs
0,user_26511551,"['video_0', 'video_0', 'video_0', 'video_0', '..."
1,user_29194819,"['video_0', 'video_0', 'video_0', 'video_0', '..."
2,user_29734049,"['video_0', 'video_0', 'video_0', 'video_0', '..."
3,user_955460,"['video_0', 'video_0', 'video_0', 'video_0', '..."
4,user_7065521,"['video_0', 'video_0', 'video_0', 'video_0', '..."
...,...,...
97235,user_29281681,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97236,user_3912848,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97237,user_28389099,"['video_0', 'video_0', 'video_0', 'video_0', '..."
97238,user_18951296,"['video_0', 'video_0', 'video_0', 'video_0', '..."


### Работа с типами данных - преобразуем 

In [10]:
for col in player_starts.select_dtypes(np.number).columns:
    player_starts[col] = pd.to_numeric(player_starts[col], downcast='integer')

In [12]:
%%time
player_starts = player_starts.drop_duplicates(subset=['date', 'user_id', 'item_id'])
player_starts.shape

CPU times: user 1min 26s, sys: 9.64 s, total: 1min 36s
Wall time: 1min 35s


(69930877, 5)

### Будем удалять все видео у которых нет watch_time

In [17]:
player_starts = player_starts[player_starts['watch_time']!=-1]

In [18]:
player_starts['date_short'] = player_starts['date'].apply(lambda l: l.split(" ")[0])
player_starts

CPU times: user 17.7 s, sys: 2.8 s, total: 20.5 s
Wall time: 20.5 s


,date,user_id,item_id,watch_time,is_autorized,date_short
0,2023-07-21 19:04:50+03:00,user_12964323,video_1042531,51,0,2023-07-21
1,2023-07-21 02:02:41+03:00,user_16517,video_1707159,31,0,2023-07-21
2,2023-07-21 22:00:47+03:00,user_15057892,video_1989987,9,0,2023-07-21
4,2023-07-21 11:06:58+03:00,user_20517034,video_1380654,11,0,2023-07-21
5,2023-07-21 23:24:41+03:00,user_8293675,video_331810,340,0,2023-07-21
...,...,...,...,...,...,...
69954375,2023-08-21 02:51:53+03:00,user_15478739,video_1449287,291,1,2023-08-21
69954376,2023-08-21 08:40:18+03:00,user_25783543,video_1423321,2,0,2023-08-21
69954377,2023-08-21 05:19:55+03:00,user_3507470,video_464555,261,0,2023-08-21
69954378,2023-08-21 12:32:32+03:00,user_13128840,video_420973,21,0,2023-08-21


In [19]:
%%time
from pandas.api.types import CategoricalDtype
users = set(pd.concat([player_starts['user_id'], sample_submission['user_id']]))
cat_users = CategoricalDtype(categories=users)
player_starts['user_id'] = player_starts['user_id'].astype(cat_users)
sample_submission['user_id'] = sample_submission['user_id'].astype(cat_users)
player_starts['item_id'] = player_starts['item_id'].astype('category')
items = player_starts['item_id'].cat.categories
index_user_id = dict(enumerate(cat_users.categories))
index_item_id = dict(enumerate(player_starts['item_id'].cat.categories))
count_user_date = player_starts.groupby(['user_id', 'date_short'])['item_id'].transform('count')
N_ITEMS_SESSION = 2
player_starts['is_active'] = count_user_date >= N_ITEMS_SESSION
player_starts_active = player_starts[player_starts['is_active']]

CPU times: user 1min 26s, sys: 6.55 s, total: 1min 33s
Wall time: 1min 33s


### Так как работать с полной adjecency матрицой неудобно, сделаем разреженную матрицу user-item

In [24]:
%%time
player_starts_sparse = csr_matrix((np.ones(player_starts_active.shape[0]), 
                                (player_starts_active.user_id.cat.codes, 
                                 player_starts_active.item_id.cat.codes)), dtype='int8')

CPU times: user 2.7 s, sys: 226 ms, total: 2.93 s
Wall time: 2.93 s


In [25]:
%%time
model = TFIDFRecommender()
model.fit(player_starts_sparse)

  0%|          | 0/2245478 [00:02<?, ?it/s]

CPU times: user 2min 51s, sys: 24 s, total: 3min 15s
Wall time: 36.8 s


Получаем рекомендации

In [26]:
%%time
N_PREDICT = 100
N_TOP = 10
FILTER_ALREADY_LIKED_ITEMS = True

users_ = sample_submission['user_id'].cat.codes.unique()
ids, scores = model.recommend(users_, player_starts_sparse.tocsr()[users_].astype(float), N=N_PREDICT, 
                              filter_already_liked_items=FILTER_ALREADY_LIKED_ITEMS)

CPU times: user 8.01 s, sys: 120 ms, total: 8.13 s
Wall time: 8.13 s


In [27]:
%%time
ids_ = ids.flatten()
rec_df = pd.DataFrame([], index=range(ids_.shape[0]), columns=['user_id', 'item_id'])
rec_df['user_id'] = np.repeat(users_, N_PREDICT)
rec_df['user_id'] = rec_df['user_id'].map(index_user_id)
rec_df['item_id'] = ids_
rec_df['item_id'] = rec_df['item_id'].map(index_item_id)
rec_df = rec_df.drop_duplicates().dropna()
rec_df

CPU times: user 17.3 s, sys: 3.26 s, total: 20.5 s
Wall time: 20.5 s


,user_id,item_id
0,user_26511551,video_1545210
1,user_26511551,video_2323123
2,user_26511551,video_885973
3,user_26511551,video_572120
4,user_26511551,video_2061037
...,...,...
9723939,user_16411220,video_844334
9723940,user_16411220,video_648740
9723941,user_16411220,video_563593
9723942,user_16411220,video_302657


Исключаем из рекоммендаций уже просмотренные пользователем видео

In [28]:
%%time
player_starts_active['user_id_item_id'] = list(zip(player_starts_active['user_id'], 
                                                   player_starts_active['item_id']))
rec_df['user_id_item_id'] = list(zip(rec_df['user_id'], rec_df['item_id']))
rec_df_ = rec_df[~rec_df['user_id_item_id'].isin(player_starts_active['user_id_item_id'])]

CPU times: user 43.4 s, sys: 4.71 s, total: 48.1 s
Wall time: 47.9 s


Некоторые пользователи остаются с менее, чем 10-тью рекомендациями или, вообще, без рекоммендаций

In [29]:
count_sample_users = len(set(sample_submission.user_id))
count_rec_users = len(set(rec_df_.user_id))
print('COUNT SAMPLE USERS =', count_sample_users)
print('COUNT REC USERS =', count_rec_users)
print('DIFF =', count_sample_users - count_rec_users)

COUNT SAMPLE USERS = 97240
COUNT REC USERS = 51777
DIFF = 45463


Рекомендуем этим пользователям самое популярное по количеству просмотров - случайным выбором

In [30]:
POPULAR_VIDEO = player_starts_active.item_id.value_counts()[:N_PREDICT].index
user_ids = sample_submission.user_id[~sample_submission.user_id.isin(rec_df_.user_id)]\
                                    .drop_duplicates().dropna().to_frame()
user_ids = pd.concat([user_ids, rec_df_.user_id.value_counts()[rec_df_.user_id.value_counts()<N_TOP]\
                      .reset_index().drop('user_id', axis=1).rename(columns={'index': 'user_id'})])                          
user_ids = pd.DataFrame(np.repeat(user_ids.values, N_PREDICT), columns=['user_id'])
user_ids['item_id'] = np.array([POPULAR_VIDEO] * (user_ids.shape[0] // N_PREDICT)).reshape(-1, 1)
user_ids = user_ids.sample(frac=1.0)
user_ids

,user_id,item_id
843522,user_29199074,video_255770
3557673,user_29206386,video_1631397
1025095,user_29436486,video_1721228
3634370,user_29222687,video_100137
2025005,user_29617700,video_68646
...,...,...
1645945,user_29244552,video_1938070
1160610,user_29427034,video_2052690
2766317,user_29163676,video_283933
4427281,user_29398420,video_965653


Исключаем из реккомендаций уже просмотренные пользователем видео и оставляем N_TOP рекомендаций

In [31]:
rec_df_['user_id_item_id'] = list(zip(rec_df_['user_id'], rec_df_['item_id']))
rec_df_ = rec_df_[~rec_df_['user_id_item_id'].isin(player_starts_active['user_id_item_id'])]

In [32]:
rec_df_ = pd.concat([rec_df_[['user_id', 'item_id']], user_ids])
rec_df_ = rec_df_.drop_duplicates().dropna()
rec_df_ = rec_df_.groupby('user_id')[['user_id', 'item_id']].head(N_TOP)
rec_df_

,user_id,item_id
0,user_26511551,video_1545210
1,user_26511551,video_2323123
2,user_26511551,video_885973
3,user_26511551,video_572120
4,user_26511551,video_2061037
...,...,...
102184,user_29673476,video_1643660
2227301,user_29367384,video_302657
2227358,user_29367384,video_1111513
279271,user_29479870,video_1997898


In [33]:
count_sample_users = len(set(sample_submission.user_id))
count_rec_users = len(set(rec_df_.user_id))
print('COUNT SAMPLE USERS =', count_sample_users)
print('COUNT REC USERS =', count_rec_users)
print('DIFF =', count_sample_users - count_rec_users)

COUNT SAMPLE USERS = 97240
COUNT REC USERS = 97240
DIFF = 0


In [34]:
rec_df_['user_id'].value_counts()

user_26511551    10
user_29446864    10
user_29268672    10
user_11091371    10
user_7337749     10
                 ..
user_6677785     10
user_24086582    10
user_13650551    10
user_954577      10
user_29494181    10
Name: user_id, Length: 97240, dtype: int64

In [36]:
features = pd.read_parquet("videos.parquet").drop_duplicates(subset=['item_id'])

CPU times: user 12.3 s, sys: 7.45 s, total: 19.7 s
Wall time: 16.9 s


In [37]:
rec_ = rec_df_.merge(features, on=['item_id'], how='left').sort_values('user_id')

In [38]:
RANDOM_USER = rec_.loc[(rec_['user_id'].isin(set(sample_submission['user_id'])))\
                       &(rec_['user_id'].isin(set(player_starts_active['user_id']))), 'user_id']\
                                                    .sample(1).values[0]
player_starts[['user_id', 'item_id']].loc[player_starts['user_id'] == RANDOM_USER]\
                                                .merge(features, on=['item_id'], how='left')

CPU times: user 17.4 s, sys: 1.87 s, total: 19.3 s
Wall time: 19.2 s


,user_id,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
0,user_19083840,video_2056378,«Братаны-2». 9 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Макар приходит к выводу, что Мила — тот челове...",Сериалы,2022-08-08 22:40:06+03:00,2836080,207061,0,0.095238,0.049383,0.083333,0.131658
1,user_19083840,video_640617,«Братаны-2». 10 серия | Боевик | Сериалы НТВ,НТВ,None,0,"К Царю приезжает зарубежный покупатель Майкл, ...",Сериалы,2022-08-08 22:45:07+03:00,2780000,207061,0,0.096774,0.084746,0.075221,0.125091
2,user_19083840,video_2056378,«Братаны-2». 9 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Макар приходит к выводу, что Мила — тот челове...",Сериалы,2022-08-08 22:40:06+03:00,2836080,207061,0,0.095238,0.049383,0.083333,0.131658
3,user_19083840,video_2056378,«Братаны-2». 9 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Макар приходит к выводу, что Мила — тот челове...",Сериалы,2022-08-08 22:40:06+03:00,2836080,207061,0,0.095238,0.049383,0.083333,0.131658
4,user_19083840,video_640617,«Братаны-2». 10 серия | Боевик | Сериалы НТВ,НТВ,None,0,"К Царю приезжает зарубежный покупатель Майкл, ...",Сериалы,2022-08-08 22:45:07+03:00,2780000,207061,0,0.096774,0.084746,0.075221,0.125091
5,user_19083840,video_542513,«Братаны-2». 14 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Макар пытается вытянуть информацию у Милы, но ...",Сериалы,2022-08-09 22:15:05+03:00,2835000,207061,0,0.142857,0.117647,0.111765,0.141743
6,user_19083840,video_1387735,«Братаны-2». 12 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Мила настаивает, что предателем мог быть и Але...",Сериалы,2022-08-09 22:05:06+03:00,2865000,207061,0,0.127660,0.094118,0.108642,0.129058
7,user_19083840,video_615608,«Братаны-2». 11 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Леший угрожает Горбу разоблачением, настаивая ...",Сериалы,2022-08-09 22:00:28+03:00,2752040,207061,0,0.200000,0.135922,0.163934,0.139771
8,user_19083840,video_615608,«Братаны-2». 11 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Леший угрожает Горбу разоблачением, настаивая ...",Сериалы,2022-08-09 22:00:28+03:00,2752040,207061,0,0.200000,0.135922,0.163934,0.139771
9,user_19083840,video_615608,«Братаны-2». 11 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Леший угрожает Горбу разоблачением, настаивая ...",Сериалы,2022-08-09 22:00:28+03:00,2752040,207061,0,0.200000,0.135922,0.163934,0.139771


In [39]:
rec_[rec_['user_id'] == RANDOM_USER]

,user_id,item_id,video_title,author_title,tv_title,season,video_description,category_title,publicated,duration,channel_sub,tv_sub,ctr.CTR_10days_21_07,ctr.CTR_10days_01_08,ctr.CTR_10days_10_08,ctr.CTR_10days_21_08
326741,user_19083840,video_85212,«Братаны-2». 25 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Макар и Ганс имитируют отравление, чтобы их пе...",Сериалы,2022-08-09 23:10:13+03:00,2824760,207061,0,0.212121,0.169014,0.136000,0.114603
326742,user_19083840,video_1186589,«Братаны-2». 26 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Локшин готовит побег Ганса и Макара, подкупает...",Сериалы,2022-08-09 23:15:06+03:00,2802960,207061,0,0.153846,0.185185,0.111111,0.093633
326744,user_19083840,video_85661,«Братаны-2». 28 серия | Боевик | Сериалы НТВ,НТВ,None,0,"С помощью Орлова, Настю отправляют на лечение ...",Сериалы,2022-08-09 23:25:07+03:00,2814800,207061,0,0.071429,0.239130,0.148515,0.133663
326740,user_19083840,video_318642,«Братаны-2». 24 серия | Боевик | Сериалы НТВ,НТВ,None,0,"Локшин встречается с начальником СИЗО, подкупа...",Сериалы,2022-08-09 23:05:11+03:00,2848040,207061,0,0.087719,0.086420,0.204082,0.101911
326743,user_19083840,video_1186910,«Братаны-2». 27 серия | Боевик | Сериалы НТВ,НТВ,None,0,Побег Макара и Ганса срывается. Серёгу задержи...,Сериалы,2022-08-09 23:20:06+03:00,2863480,207061,0,0.078947,0.268293,0.165217,0.105691
326746,user_19083840,video_2011718,«Братаны-2». 31 серия | Боевик | Сериалы НТВ,НТВ,None,0,Локшин быстро берет власть над организацией Ко...,Сериалы,2022-08-09 23:40:07+03:00,2858640,207061,0,0.114286,0.125000,0.149425,0.142445
326747,user_19083840,video_1990295,Время приключений\nAdventure.Time.S07E20.Bad.J...,watch a movie,None,0,Год производства\n2010 (10 сезонов)\nСтрана\nС...,Сериалы,2023-08-10 11:53:36+03:00,686113,123,0,NaN,NaN,0.000000,NaN
326748,user_19083840,video_914701,"Лучшие сериалы 2022 , которые УЖЕ ВЫШЛИ | ТОП ...",Кинотоп | КиноЧерника,None,0,В этом ролике мы собрали лучшие (по мнению зри...,Сериалы,2022-06-27 04:16:51+03:00,651924,1654,0,0.000000,0.000000,0.000000,0.000000
326749,user_19083840,video_976325,"Острые Козырьки 3 сезон, 2 серия смотреть бесп...",Сериал Острые козырьки,None,0,#ОстрыеКозырьки #детектив #бандиты #криминал #...,Сериалы,2023-08-07 11:40:17+03:00,3491480,178,0,NaN,NaN,NaN,0.000000
326745,user_19083840,video_680566,«Братаны-2». 30 серия | Боевик | Сериалы НТВ,НТВ,None,0,Братаны договариваются с Орловым встретиться в...,Сериалы,2022-08-09 23:35:07+03:00,2854320,207061,0,0.179487,0.365854,0.245614,0.162861


# Metrics

In [ ]:
item_genres_one_hot = videos[["item_id", "category_title"]].copy()
item_genres_one_hot["category_title"] = item_genres_one_hot["category_title"].str.split(", ")
item_genres_one_hot = item_genres_one_hot.explode("category_title")
item_genres_one_hot["category_title"] = item_genres_one_hot["category_title"].str.replace(" ", "_")
item_genres_one_hot["category_title"] = item_genres_one_hot["category_title"].map(lambda x: translit(x, "ru", reversed=True))
item_genres_one_hot["value"] = 1
item_genres_one_hot = item_genres_one_hot.pivot(
    index="item_id", 
    columns="category_title", 
    values="value"
).fillna(0).astype(int)

item_genres_one_hot.head()

In [ ]:
def get_hamming_distances(pairs: pd.Series, features: pd.DataFrame) -> np.ndarray:
    items_0 = pairs.map(lambda pair: pair[1]).values
    items_1 = pairs.map(lambda pair: pair[0]).values

    features_0 = features.reindex(items_0).values
    features_1 = features.reindex(items_1).values
    return np.sum(features_0 != features_1, axis=1)


def calculate_intra_list_diversity_per_user(recommendations: pd.DataFrame, features: pd.DataFrame) -> pd.Series:
    recommended_item_pairs = recommendations.groupby("user_id")["item_id"].apply(
        lambda x: list(combinations(x, 2))
    ).reset_index().explode("item_id").rename(columns={"item_id": "item_pair"})
    recommended_item_pairs["dist"] = get_hamming_distances(recommended_item_pairs["item_pair"], features)
    return recommended_item_pairs[["user_id", "dist"]].groupby("user_id").agg("mean")

In [ ]:
for model_name, recommendations in recommendations_dict.items():
    ild_per_user = calculate_intra_list_diversity_per_user(recommendations, item_genres_one_hot)
    print(f"model: {model_name}, mean ild: {round(float(ild_per_user.mean()), 2)}\n")

# SUBMIT

In [40]:
RECS = rec_df_.groupby('user_id')['item_id'].apply(lambda x: list(x.values)).reset_index()

In [41]:
submission = sample_submission.drop('recs', axis=1).copy()
submission = submission.merge(RECS, on='user_id').rename(columns={'item_id': 'recs'})
submission

,user_id,recs
0,user_26511551,"[video_1545210, video_2323123, video_885973, v..."
1,user_29194819,"[video_836422, video_2052690, video_318057, vi..."
2,user_29734049,"[video_1938070, video_1582799, video_1901127, ..."
3,user_955460,"[video_2336686, video_1979002, video_1643660, ..."
4,user_7065521,"[video_17675, video_2088291, video_890608, vid..."
...,...,...
97235,user_29281681,"[video_814938, video_1250322, video_433884, vi..."
97236,user_3912848,"[video_165745, video_1125654, video_751399, vi..."
97237,user_28389099,"[video_902590, video_302657, video_889838, vid..."
97238,user_18951296,"[video_2323393, video_1729455, video_1072584, ..."


In [42]:
submission.to_csv("submission_READY.csv", index=False)